<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">

In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em ordem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '10402412, Diego Oliveira Aluizio' #@param {type:"string"}
Aluno2 = '10396490, Jônatas Garcia de Oliveira' #@param {type:"string"}
Aluno3 = '10403046, Livia Alabarse dos Santos' #@param {type:"string"}
Aluno4 = '10403028, Marina Scabello Martin' #@param {type:"string"}
Aluno5 = '10265432, Pedro Henrique Araujo Farias' #@param {type:"string"}


In [ ]:
#@title Assinale aqui a sua opção de Projeto
Projeto = "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos" #@param ["IA Aplicada a Imagens: Uso de Modelos de Redes Neurais", "IA Aplicada a Documentos: Uso de Grandes Modelos de Linguagem Abertos"]




# **Resumo**

O objetivo deste *notebook* é desenvolver o treinamento de um modelo para a classificação de textos com base em seu sentimento, podendo ser classificados como **positivo**, **neutro** ou **negativo**, tarefa conhecida como **análise de sentimento**. Os dados utilizados para treinamento do modelo são uma amostra do *dataset* **Tweet_Eleições_2022**. As bibliotecas e ferramentas de IA utilizadas foram a ***Transformers***, fornecida pela ***Hugging Face***, e o ***TensorFlow***, fornecido pela ***Google***. Como resultado, obtivemos um modelo com 89,85% de acurácia nos testes, que foi disponibilizado na plataforma do *Hugging Face*.

# **Bibliotecas utilizadas**

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, create_optimizer

# **Apresentação dos dados**

Os dados utilizados foram obtidos a partir do *dataset* [Tweet_Eleições_2022](https://github.com/ciberdem/Tweets_Eleicoes_2022) (SILVA *et al*., 2024), que reúne aproximadamente 9,5 milhões de tweets coletados ao longo do processo eleitoral brasileiro de 2022, via API oficial do *X*, antigo *Twitter*.

Os dados foram pré-processados a partir das seguintes atividades: remoção de duplicatas, remoção de ruídos e anonimização de usuários, bem como seleção temporal, restringindo o conjunto à *tweets* publicados no dia 8 de janeiro de 2023. O resultado do pré-processamento foi um conjunto de 984 *tweets*.

Os integrantes do projeto realizaram anotação manual dos dados de acordo com o sentimento de cada *tweet* analisado. Você pode acessar a planilha de dados utilizados para treinamento neste [link](https://docs.google.com/spreadsheets/d/1wklfGXQpK6i5W9hOcYyIxT1BcdkuN-Ik/edit?usp=drive_link&ouid=102543127324810311691&rtpof=true&sd=true).

In [ ]:
import pandas as pd

tweets_file_id = "1wklfGXQpK6i5W9hOcYyIxT1BcdkuN-Ik"
url_tweets = f"https://drive.google.com/uc?id={tweets_file_id}"

df = pd.read_excel(url_tweets)
display(df)

# **Preparação e transformação dos dados**

As únicas colunas interessantes para treinamento do modelo são `text` e `Sentimento`, portanto a coluna `conversation_id` será **removida**.

In [ ]:
df = df.drop('conversation_id', axis=1)
display(df)
print(f"\nDistribuição dos sentimentos:\n{df['Sentimento'].value_counts()}")

Em seguida, realizamos o ***label encoding*** da coluna `Sentimento`, a qual representa nossa variável objetivo.

In [ ]:
label_map = {"POSITIVO": 0, "NEUTRO": 1, "NEGATIVO": 2}
df['sentimento'] = df['Sentimento'].map(label_map)
df = df.drop('Sentimento', axis=1)
display(df)

Por fim, vamos separar os dados em **três conjuntos**:
- **Treinamento**: utilizados para aprendizagem do modelo durante o treinamento;
- **Validação**: utilizados para validação da aprendizagem do modelo durante o treinamento;
- **Teste**: utilizados para avaliação do modelo após o treinamento.

In [ ]:
train_val_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['sentimento']
)

train_df, val_df = train_test_split(
    train_val_df, test_size=0.25, random_state=42, stratify=train_val_df['sentimento']
)

In [ ]:
print(f"Tamanho do conjunto de Treino: {len(train_df)}")
display(train_df)

print(f"Tamanho do conjunto de Validação: {len(val_df)}")
display(val_df)

print(f"Tamanho do conjunto de Teste: {len(test_df)}")
display(test_df)

# **Configuração e Treinamento do Modelo**

## **Modelo pré-treinado**

O modelo pré-treinado utilizado para desenvolvimento do modelo deste *notebook* é o **BERTimbau Base**, um modelo baseado em BERT para análise de linguagem natural em português brasileiro.

## **BERTimbau: *Tokenizer* e *Encoding* dos dados**

Para utilizar o BERTimbau, precisamos utilizar também seu ***tokenizer*** para **converter texto em sequências de tokens** que o modelo BERTimbau entende, e vice-versa:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

A função `encode_texts` utiliza o ***tokenizer*** para preparar os dados para entrada no modelo BERTimbau, garantindo que todos tenham um comprimento fixo (max_len). Note que passamos `return_tensors='tf'` como argumento pois estamos utilizando o **TensorFlow**.

In [ ]:
def encode_texts(tokenizer, texts, max_len):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

Aplicando o *encoding* aos conjuntos:

In [ ]:
MAX_LEN = 128

train_x = encode_texts(tokenizer, train_df['text'], MAX_LEN)
val_x = encode_texts(tokenizer, val_df['text'], MAX_LEN)
test_x = encode_texts(tokenizer, test_df['text'], MAX_LEN)

Transformando os dados e rótulos em pipelines de dados `tf.data` para serem alimentados no treinamento e avaliação pelo `TensorFlow`, embaralhando o conjunto de treinamento:

In [ ]:
train_y = train_df['sentimento'].to_numpy()
val_y = val_df['sentimento'].to_numpy()
test_y = test_df['sentimento'].to_numpy()

In [ ]:
train_dataset_dict = {key: train_x[key] for key in ['input_ids', 'attention_mask']}
val_dataset_dict = {key: val_x[key] for key in ['input_ids', 'attention_mask']}
test_dataset_dict = {key: test_x[key] for key in ['input_ids', 'attention_mask']}

In [ ]:
BATCH_SIZE = 32

train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_dataset_dict, train_y)).shuffle(len(train_df)).batch(BATCH_SIZE)
val_tf_dataset = tf.data.Dataset.from_tensor_slices((val_dataset_dict, val_y)).batch(BATCH_SIZE)
test_tf_dataset = tf.data.Dataset.from_tensor_slices((test_dataset_dict, test_y)).batch(BATCH_SIZE)

Note duas operações essenciais:
- `input_ids`: Mapeando para os tensores dos IDs dos tokens.
- `attention_mask`: Mapeando para os tensores das máscaras de atenção.

Esses mapeamentos são extremamente importantes para que o modelo compreenda e processe os dados.

## **BERTimbau: Carregando o Modelo**

Carregamos o modelo BERTimbau (`"neuralmind/bert-base-portuguese-cased"`) e definimos um **novo modelo**, utilizando o BERTimbau como modelo pré-treinado, adicionando uma camada de classificação no topo, ainda não treinada, com apenas **3 classificações possíveis** (`num_labels=3`), as quais classificarão o sentimento do texto analisado.

In [ ]:
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"
id_to_label = {v: k for k, v in label_map.items()}

model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=id_to_label,
    label2id=label_map
)

Em seguida, **congelamos as camadas de *embedding*** e **8 camadas de *encoder*** do BERTimbau com **objetivo de impedir que os pesos dessas camadas sejam atualizados durante o processo de *fine-tuning***, preservando o conhecimento do BERTimbau e reduzindo o número de parâmetros a serem treinados, o que torna o treinamento deste modelo mais rápido.

In [ ]:
bert_main_layer = model.layers[0]
bert_main_layer.embeddings.trainable = False

NUM_LAYERS_TO_FREEZE = 8

if NUM_LAYERS_TO_FREEZE > 0 and hasattr(bert_main_layer, 'encoder'):
    print(f"Congelando as primeiras {NUM_LAYERS_TO_FREEZE} camadas do encoder BERT.")
    for i in range(NUM_LAYERS_TO_FREEZE):
        if i < len(bert_main_layer.encoder.layer):
            bert_main_layer.encoder.layer[i].trainable = False
            print(f"Encoder layer {i} congelada: {not bert_main_layer.encoder.layer[i].trainable}")
        else:
            print(f"Aviso: Tentativa de congelar camada {i}, mas o encoder tem apenas {len(bert_main_layer.encoder.layer)} camadas.")
            break
else:
    print("Nenhuma camada do encoder para congelar ou encoder não encontrado como esperado.")

Definindo alguns **hiperparâmetros** do modelo, como **número de épocas**, **taxa de aprendizagem** e **otimizador** utilizados:

In [ ]:
EPOCHS = 10
LEARNING_RATE = 2e-5

num_train_steps = (len(train_df) // BATCH_SIZE) * EPOCHS

optimizer, schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01
)

Agora, vamos definir a função de perda a ser minimizada e sua métrica de desempenho. Por fim, compilamos o modelo.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)
model.summary()

Obs.: Algo importante de se comentar é o número de parâmetros na camada `dropout_75`. Essa camada não apresenta parâmetros a serem treinados, pois seu único propósito é **desligar aleatoriamente alguns neurônios da rede, evitando *overfitting***.

Como estávamos enfrentando **sérios problemas com *overfitting* durante o treinamento do modelo**, definimos também o ***early_stopping* e seus parâmetros**:

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

## **Treinamento do Modelo**

Vamos, finalmente, treinar nosso modelo!

Explicando brevemente os argumentos utilizados na função `fit`:
- `train_df_dataset`: Utilizamos o conjunto de **treinamento** como *dataset* normalizado para treinamento pelo modelo;
- `validation_data=val_tf_dataset`: Utilizamos o `val_tf_dataset` como *dataset* normalizado para validação do treinamento do modelo;
- `epochs=EPOCHS`: Definimos o número de épocas, estabelecido como **10 épocas**;
- `batch_size=BATCH_SIZE`: Definimos o tamanho do lote para treinamento;
- `callbacks=[early_stopping]`: Passamos a nossa função de *Early Stopping* definida na célula anterior para evitar *overfitting*.

In [ ]:
history = model.fit(
    train_tf_dataset,
    validation_data=val_tf_dataset,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping]
)

# **Avaliação do modelo**



Com base nas informações de treinamento armazenadas na variável `history`, vamos plotar gráficos de *loss* e *acurácia* para avaliar o treinamento do modelo:

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss Treino')
plt.plot(history.history['val_loss'], label='Loss Validação')
plt.title('Histórico de Loss')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Acurácia Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia Validação')
plt.title('Histórico de Acurácia')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)

plt.show()

Como é possível observar, o modelo apresenta ***loss* decrescente** para o **conjunto de treinamento**. Quanto ao **conjunto de validação**, a ***loss* decai até a sexta época**, onde acaba **estagnando até o fim do treinamento** (alerta para possível *overfitting*).

Quanto à acurácia, esta é **crescente tanto para o conjunto de treinamento quanto para o de validação**. Curiosamente, a **acurácia do conjunto de validação parte de valores muito altos**, iniciando acima de 80%. A mesma tendência a *overfitting* se observa no gráfico de acurácia.

Ainda que essa tendência seja recorrente, é importante observar que sua magnitude não se demonstra alarmante. Sendo assim, não é um modelo perfeito, mas também não é um modelo péssimo.

Dado a pequena quantidade de dados da amostra, consideramos satisfatórios os resultados apresentados nos gráficos de *loss* e *accuracy*;

In [ ]:
test_loss, test_accuracy = model.evaluate(test_tf_dataset, verbose=1)
print(f"Acurácia no Teste: {test_accuracy:.4f}")
print(f"Loss no Teste: {test_loss:.4f}")

In [ ]:
predictions = model.predict(test_dataset_dict, batch_size=BATCH_SIZE)
predicted_logits = predictions.logits
predicted_labels_int = np.argmax(predicted_logits, axis=1)
target_names = [id_to_label[i] for i in range(3)]
print(classification_report(test_y, predicted_labels_int, target_names=target_names, digits=4))

Quanto ao *classification report*, o modelo apresenta bons valores para as métricas de *precision* e *recall* (> 80%). O ***f1-score***, particularmente, se mantém **acima de 85%**.

# **Consumo do Modelo**

## **Salvando o Modelo**

Para consumir o modelo em uma aplicação, inicialmente salvamos o modelo e realizamos o *download* dos arquivos com a intenção de realizar o *upload* no ***GitHub***, de modo que a aplicação acessasse o modelo localmente:

In [ ]:
OUTPUT_MODEL_DIR = "./modelo_sentimento_bertimbau_finetuned_tf"

model.save_pretrained(OUTPUT_MODEL_DIR)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)
print(f"Modelo e tokenizador salvos em: {OUTPUT_MODEL_DIR}")

Entretanto, por ser um arquivo muito grande, o **modelo não pôde ser enviado ao *GitHub***. Sendo assim, optamos por publicar o modelo na plataforma do ***Hugging Face***:

In [ ]:
!pip install -q huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
repo_id = "chrnphxbia/djmlp_tiny_analise_sentimento"

model.push_to_hub(
    repo_id,
    commit_message="Upload do modelo via Colab",
    private=False,
    create_repo=True
)

tokenizer.push_to_hub(
    repo_id,
    commit_message="Update do tokenizer via Colab",
    private=False
)

print(f"Modelo e tokenizador enviados para: https://huggingface.co/{repo_id}")

## **Aplicação *Streamlit* para consumo do Modelo**

Desenvolvemos uma **aplicação *Streamlit*** que consome o modelo. Na aplicação, o usuário pode **inserir um texto para ser analisado**, ou **enviar um arquivo .xlsx ou .csv para analisar cada um dos textos do arquivo**, devendo **selecionar a coluna que apresenta os textos a serem analisados**.

Ao final da execução, **a aplicação apresenta o sentimento do texto analisado** e, caso tenha sido enviado um arquivo, **a aplicação permite que o usuário faça *download* de uma planilha com a coluna de sentimento definido adicionada**.

A aplicação foi hospedada no ***Streamlit Cloud*** e pode ser acessada neste [*link*](https://djlmpsentimentanalyzer.streamlit.app/).

O código-fonte da aplicação está hospedado no [repositório do projeto](https://github.com/chrnphxbia/djlmp_sentiment_analyzer).

# **Referências**

- Abadi, Martín et al. TensorFlow: Large-Scale Machine Learning on Heterogeneous Systems. 2015. Disponível em: https://www.tensorflow.org/. Acesso em: 01 jun. 2025.

- HUGGING FACE. Compartilhando modelos pré-treinados. Disponível em: https://huggingface.co/learn/llm-course/pt/chapter4/3. Acesso em: 1 jun. 2025.

- HUGGING FACE. Tokenizer. Disponível em: https://huggingface.co/docs/transformers/main_classes/tokenizer. Acesso em: 1 jun. 2025.

- HUGGING FACE. Uploading models. Disponível em: https://huggingface.co/docs/hub/models-uploading. Acesso em: 1 jun. 2025.

- SILVA, Luciano José da et al. Tweet_Eleições_2022: Um dataset de tweets durante as eleições presidenciais brasileiras de 2022. In: BRAZILIAN WORKSHOP ON SOCIAL NETWORK ANALYSIS AND MINING (BRASNAM), 13., 2024, Brasília/DF. Anais [...]. Porto Alegre: Sociedade Brasileira de Computação, 2024. p. 193-199. DOI 10.5753/brasnam.2024.1940. Disponível em: https://sol.sbc.org.br/index.php/brasnam/article/view/29343. Acesso em: 30 maio 2025.

- SOUZA, Fábio. neuralmind/bert-base-portuguese-cased. [S. l.]: Hugging Face, 2020. Disponível em: https://huggingface.co/neuralmind/bert-base-portuguese-cased. Acesso em: 1 jun. 2025.

- TALEBI, S. Fine-Tuning BERT for Text Classification (w/ Example Code). [Publicado em 17 de out. de 2024]. Disponível em: http://www.youtube.com/watch?v=4QHg8Ix8WWQ. Acesso em: 1 de jun. de 2025.

- WOLF, Thomas et al. HuggingFace's Transformers: State-of-the-art Natural Language Processing. CoRR, v. abs/1910.03771, 2019. Disponível em: http://arxiv.org/abs/1910.03771. Acesso em: 1 jun. 2025.





---

In [ ]:
# @title **Avaliação**
GitHub = 10 #@param {type:"slider", min:0, max:10, step:1}

Implementacao_Model_Code = 7 #@param {type:"slider", min:0, max:10, step:1}

Aplicacao_Streamlit = 9 #@param {type:"slider", min:0, max:10, step:1}

Texto_Artigo  = 6 #@param {type:"slider", min:0, max:10, step:1}

Video = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**

nota = 2*GitHub + 4*Implementacao_Model_Code + 2*Aplicacao_Streamlit + 1*Texto_Artigo + 1*Video

nota = nota / 10

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)